In [1]:
import torch

In [3]:
a=torch.randint(1, 54999, (500,))
b=torch.rand((500,))
a1=torch.randint(1, 54999, (500,))
b1=torch.rand((500,))

def f(a, b, a1, b1):
    v=torch.zeros((55000))
    v[a] = b
    v1=torch.zeros((55000))
    v1[a1]=b1
    return torch.inner(v, v1)
f(a, b, a1, b1)
# timeit f(a, b, a1, b1)

def f1(a, b, a1, b1):
    k = {}
    for ia, ib in zip(a, b):
        k[int(ia)] = float(ib)
    v=0
    for ia, ib in zip(a1, b1):
        v += k.get(int(ia), 0)*float(ib)
    return v


In [4]:
%timeit f1(a, b, a1, b1)



1.46 ms ± 4.31 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
109 μs ± 909 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [5]:
%timeit f(a, b, a1, b1)

97.4 μs ± 15.3 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [19]:
def f2(a, b, a1, b1):
    v=torch.zeros((100, 55000), device='cuda')
    for i in range(a.size()[0]):
        v[i][a[i]] = b[i]
    v1=torch.zeros((100, 55000), device='cuda')
    for i in range(a1.size()[0]):
        v1[i][a[i]] = b[i]
    return torch.inner(v, v1)


In [7]:
def create_int(n1, n2=1):
    if n2==1:
        return torch.randint(1, 54999, (n1,), device='cuda')
    else:
        return torch.randint(1, 54999, (n1, n2), device='cuda')

In [8]:
def create_rand(n1, n2=1):
    if n2==1:
        return torch.rand((n1), device='cuda')
    else:
        return torch.rand((n1, n2), device='cuda')

In [9]:
aa=create_int(100, 500)
aa1=create_int(100, 500)
bb=create_rand(100, 500)
bb1=create_rand(100, 500)

In [15]:
v=torch.zeros((100, 55000), device='cuda')

In [16]:
v[0][a[0]]=b[0]

In [14]:
v[0].size()

torch.Size([100])

In [20]:
f2(aa, bb, aa1, bb1)

tensor([[161.0678,   1.4728,   1.5088,  ...,   1.1825,   0.3160,   1.2653],
        [  1.4728, 162.9776,   2.5915,  ...,   0.3478,   1.9206,   1.8399],
        [  1.5088,   2.5915, 164.5229,  ...,   1.3973,   2.0075,   0.5757],
        ...,
        [  1.1825,   0.3478,   1.3973,  ..., 165.5163,   0.6630,   0.8933],
        [  0.3160,   1.9206,   2.0075,  ...,   0.6630, 161.7457,   1.3474],
        [  1.2653,   1.8399,   0.5757,  ...,   0.8933,   1.3474, 176.7607]],
       device='cuda:0')

In [21]:
%timeit f2(aa, bb, aa1, bb1)

1.64 ms ± 5.12 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
